In [14]:
import torch as tr
import torchvision as tv
import pandas as pd
import numpy as np
import scipy as scp
import datetime as dt
from torchvision.transforms import v2
import cv2

from sklearn.metrics import accuracy_score, precision_score, recall_score, mean_squared_error
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

from torch.utils.data import Dataset, DataLoader
import torch.nn as nn

from matplotlib import pyplot as plt
import seaborn as sns

import os

In [15]:
!tree -L 2

.
├── baseline-in-pytorch.ipynb
├── lab_1.ipynb
├── lab_2.ipynb
├── lab_3.ipynb
├── lab_4.ipynb
├── lab_5_data
├── lab_5_data_raw
│   ├── __MACOSX
│   ├── plates
│   ├── platesv2.zip
│   ├── plates.zip
│   └── sample_submission.csv
├── lab_5.ipynb
├── lab_5_models_meta
│   ├── 2024-03-11 13:05:21.176739
│   └── 2024-03-11 19:10:47.357940
├── README.md
├── submission.csv
├── test
│   └── unknown
├── train
│   ├── cleaned
│   └── dirty
└── val
    ├── cleaned
    └── dirty

13 directories, 13 files


In [16]:
os.listdir("./lab_5_data/plates/train")

FileNotFoundError: [Errno 2] No such file or directory: './lab_5_data/plates/train'

In [137]:
data_root = "./lab_5_data/plates/"
train_dir = os.path.join(data_root, "train")

train_dirty_dir = os.path.join(train_dir, "dirty")
train_cleaned_dir = os.path.join(train_dir, "cleaned")

test_dir = os.path.join(data_root, "test")
# os.listdir(data_root)
# os.listdir(train_dir)
# os.listdir(train_dirty_dir)
# os.listdir(train_cleaned_dir)
# os.listdir(test_dir)

In [13]:
img = cv2.imread(
    os.path.join(
        train_dirty_dir, os.listdir(train_dirty_dir)[1]
    ))
transforms_t = v2.Compose([
    
    v2.ToImage(),
    v2.ToDtype(tr.float32, scale=True),
    
    v2.ColorJitter(
        brightness  = ( 1, 2 ),
        contrast    = ( 0.1, 1.3 ),    
        saturation  = ( 2 , 10),      #grayscale to pure color
        hue         = ( -0.09 , 0.09  ),  #actual color
    ),
    
    # v2.CenterCrop(200),
    
    # v2.RandomRotation(degrees=(5, 10)),
    # v2.RandomHorizontalFlip(0.6),
    # v2.RandomVerticalFlip(0.6),
    # v2.RandomPerspective(
    #     distortion_scale=0.2,
    #     p=0.7
    # ),
    
    # v2.RandomErasing(
    #     p=0,
    #     scale=(0.1, 0.1),
    #     ratio=(0.1, 0.1),
    #     value=1,
    #     inplace=False
    # ),
    
    # v2.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    
    v2.Lambda( lambda pic: tr.clip(pic,0,1) ),
    v2.Resize(size=(256, 341)),
    
])

tmp = transforms_t(img).permute(1,2,0)
fig,ax = plt.subplots(1,2)
ax[0].imshow(img)
ax[0].set_title("BEFORE")
ax[1].imshow(tmp)
ax[1].set_title("AFTER")
print(img.dtype, tmp.dtype)
print(img.min(), tmp.min())
print(img.max(), tmp.max())

NameError: name 'cv2' is not defined

In [351]:
transforms = v2.Compose([
    
    v2.ToImage(),
    v2.ToDtype(tr.float32, scale=True),
    
    v2.ColorJitter(
        brightness  = ( 1, 2 ),
        contrast    = ( 0.1, 1.3 ),    
        saturation  = ( 2 , 10),      #grayscale to pure color
        # hue         = ( -0.09 , 0.09  ),  #actual color
    ),
    
    v2.CenterCrop(250),
    
    v2.RandomRotation(degrees=(5, 10)),
    v2.RandomHorizontalFlip(0.6),
    v2.RandomVerticalFlip(0.6),
    v2.RandomPerspective(
        distortion_scale=0.2,
        p=0.7
    ),
    
    v2.RandomErasing(
        p=0.3,
        scale=(0.1, 0.1),
        ratio=(0.1, 0.1),
        value=1,
        inplace=False
    ),
    
    v2.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    
    v2.Lambda( lambda pic: tr.clip(pic,0,1) ),
    v2.Resize(size=(256, 341)),
    

])

transforms1 = v2.Compose([
    
    v2.ToImage(),
    v2.ToDtype(tr.float32, scale=True),
    
    v2.ColorJitter(
        brightness  = ( 1, 2 ),
        contrast    = ( 0.1, 1.3 ),    
        saturation  = ( 2 , 10),      #grayscale to pure color
        # hue         = ( -0.09 , 0.09  ),  #actual color
    ),
    
    v2.CenterCrop(250),
    
    v2.RandomRotation(degrees=(5, 10)),
    # v2.RandomHorizontalFlip(0.6),
    v2.RandomVerticalFlip(0.6),
    v2.RandomPerspective(
        distortion_scale=0.2,
        p=0.7
    ),
    
    v2.RandomErasing(
        p=0.5,
        scale=(0.1, 0.1),
        ratio=(0.1, 0.1),
        value=1,
        inplace=False
    ),
    
    v2.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    
    v2.Lambda( lambda pic: tr.clip(pic,0,1) ),
    v2.Resize(size=(256, 341)),
    

])

transforms2 = v2.Compose([
    
    v2.ToImage(),
    v2.ToDtype(tr.float32, scale=True),
    
    v2.ColorJitter(
        brightness  = ( 1, 2 ),
        contrast    = ( 0.1, 1.3 ),    
        saturation  = ( 2 , 10),      #grayscale to pure color
        # hue         = ( -0.09 , 0.09  ),  #actual color
    ),
    
    # v2.CenterCrop(250),
    
    # v2.RandomRotation(degrees=(5, 10)),
    # v2.RandomHorizontalFlip(0.6),
    # v2.RandomVerticalFlip(0.6),
    # v2.RandomPerspective(
    #     distortion_scale=0.2,
    #     p=0.7
    # ),
    
    v2.RandomErasing(
        p=0.5,
        scale=(0.1, 0.1),
        ratio=(0.1, 0.1),
        value=1,
        inplace=False
    ),
    
    v2.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    
    v2.Lambda( lambda pic: tr.clip(pic,0,1) ),
    v2.Resize(size=(256, 341)),
    

])

transforms3 = v2.Compose([
    
    v2.ToImage(),
    v2.ToDtype(tr.float32, scale=True),
    
    v2.ColorJitter(
        brightness  = ( 1, 2 ),
        contrast    = ( 0.1, 1.3 ),    
        saturation  = ( 2 , 10),      #grayscale to pure color
        # hue         = ( -0.09 , 0.09  ),  #actual color
    ),
    
    v2.CenterCrop(250),
    
    v2.RandomRotation(degrees=(5, 10)),
    v2.RandomHorizontalFlip(0.6),
    v2.RandomVerticalFlip(0.6),
    v2.RandomPerspective(
        distortion_scale=0.2,
        p=0.7
    ),
    
    # v2.RandomErasing(
    #     p=0.5,
    #     scale=(0.1, 0.1),
    #     ratio=(0.1, 0.1),
    #     value=1,
    #     inplace=False
    # ),
    
    v2.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    
    v2.Lambda( lambda pic: tr.clip(pic,0,1) ),
    v2.Resize(size=(256, 341)),
    

])

In [364]:
from PIL import Image


# os.makedirs("extra/dirty")
# os.makedirs("extra/cleaned")

ident = np.random.randint(-100000,100000)

for ind, img_path in enumerate(os.listdir(train_cleaned_dir)):
    img = Image.open(
        os.path.join(
            train_cleaned_dir, img_path
        ))
    tr_img = transforms(img)
    tv.utils.save_image(tr_img, f"extra/cleaned/{ident}011100{ind+0}.jpg")


for ind, img_path in enumerate(os.listdir(train_cleaned_dir)):
    img = Image.open(
        os.path.join(
            train_cleaned_dir, img_path
        ))
    tr_img = transforms1(img)
    tv.utils.save_image(tr_img, f"extra/cleaned/{ident}00{ind+100}.jpg")
    
for ind, img_path in enumerate(os.listdir(train_cleaned_dir)):
    img = Image.open(
        os.path.join(
            train_cleaned_dir, img_path
        ))
    tr_img = transforms2(img)
    tv.utils.save_image(tr_img, f"extra/cleaned/{ident}00{ind+200}.jpg")
    
for ind, img_path in enumerate(os.listdir(train_cleaned_dir)):
    img = Image.open(
        os.path.join(
            train_cleaned_dir, img_path
        ))
    tr_img = transforms3(img)
    tv.utils.save_image(tr_img, f"extra/cleaned/{ident}00{ind+300}.jpg")


###dirty 
for ind, img_path in enumerate(os.listdir(train_dirty_dir)):
    img = Image.open(
        os.path.join(
            train_dirty_dir, img_path
        ))
    tr_img = transforms(img)
    tv.utils.save_image(tr_img, f"extra/dirty/{ident}11100{ind}.jpg")


for ind, img_path in enumerate(os.listdir(train_dirty_dir)):
    img = Image.open(
        os.path.join(
            train_dirty_dir, img_path
        ))
    tr_img = transforms1(img)
    tv.utils.save_image(tr_img, f"extra/dirty/{ident}00{ind+100}.jpg")
    
for ind, img_path in enumerate(os.listdir(train_dirty_dir)):
    img = Image.open(
        os.path.join(
            train_dirty_dir, img_path
        ))
    tr_img = transforms2(img)
    tv.utils.save_image(tr_img, f"extra/dirty/{ident}00{ind+200}.jpg")
    
for ind, img_path in enumerate(os.listdir(train_dirty_dir)):
    img = Image.open(
        os.path.join(
            train_dirty_dir, img_path
        ))
    tr_img = transforms3(img)
    tv.utils.save_image(tr_img, f"extra/dirty/{ident}00{ind+300}.jpg")

In [11]:
! rm -rf extra
! mkdir extra
! mkdir extra/dirty/
! mkdir extra/cleaned/
! find .  -type d -name ".ipynb_checkpoints" -print0 -exec rm -rf {} +
! rm -rf lab_5_data/plates/

./.ipynb_checkpoints ./lab_5_data/.ipynb_checkpoints 

In [12]:
! mkdir lab_5_data/plates

In [10]:
! cp -r lab_5_data/plates_if_repeat/ lab_5_data/plates/

In [365]:
! mv extra/cleaned/* lab_5_data/plates/train/cleaned
! mv extra/dirty/* lab_5_data/plates/train/dirty